# Setup

## Imports

In [52]:
import os

import pandas
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

## Constants

In [53]:
numeric_columns = ['Age', 'Tscore', 'Height', 'Weight', 'BMI', 'BASP', 'PTHpmoll', 'Vitdnmoll', 'CTXmygl', 'P1NP', 'calciumion', 'creatinin', 'eGFR', 'oks_total', 'VAS_aktiv']

boolean_columns = ['Sex', 'Cem_ucem', 'TKA', 'side', 'exercise', 'former_alcoholabuse', 'smoker', 'former_smoker']

# Setup Data

## Load Data

In [54]:
# Train data
train_data_path = "data/Train_data.xls"
train_X = pandas.read_excel(train_data_path)
train_X.pop('Number')
train_y = train_X.pop('MIG_group')

# Test data
test_data_path = "data/Test_data.xls"
test_X = pandas.read_excel(test_data_path)
test_X.pop('Number')

# Validation data
validation_data_path = "data/Validation_data.xls"
validation_X = pandas.read_excel(validation_data_path)
validation_X.pop('Number')
validation_y = validation_X.pop('MIG_group')

## Fix absent data
- take mean value for mean values
- take most popular for booleans

In [55]:
# Fixes data
# When a value is missing
# - if column is numeric -> take mean
# - if column is boolean -> take most popular
def fixAbsentData(df: pd.DataFrame):

    # Get Values for numeric columns
    for column in numeric_columns:
        if column in df:
            mean = df[column].mean()
            df[column] = df[column].fillna(mean)

    # Get Values for boolean columns
    for column in boolean_columns:
        if column in df:
            mean = df[column].mean()

            if(mean <= 0.5):
                mean = 0
            else:
                mean = 1
            df[column] = df[column].fillna(mean)
    return df

train_X = fixAbsentData(train_X)
test_X = fixAbsentData(test_X)
validation_X = fixAbsentData(validation_X)

# Model Training

# Data Visualization